<a href="https://colab.research.google.com/github/jdmiranda/ai_notebooks/blob/master/a_star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robot Navigation Problem
# Uses Algorithm A* to find the shortest path around obstacles

In [0]:
import heapq  # for priority queue
from time import time # for run times
import matplotlib.pyplot as plt # for displaying solutions
import pandas as pd # to save results

In [0]:
dataDir, outDir = './', './'
#dataDir, outDir = 'data/', 'out/' # directories for input/output files

#inputFiles = ['robot'+str(i)+'.txt' for i in range(1,16)] # input files
inputFiles = ['robot1.txt']

In [0]:
def readProblem(filename):
    """Reads input file and returns start, goal, and
    a dictionary containing polygons as a list of vertices"""
    f = open(filename)
    start = tuple([int(v) for v in f.readline().split(',')])
    goal = tuple([int(v) for v in f.readline().split(',')])
    nPoly = int(f.readline())
    polygon = {}
    for i in range(nPoly): # read vertices of polygons
        line = [int(v) for v in f.readline().split(',')]
        polygon[i+1] = [(line[2*j+1],line[2*j+2]) for j in range(line[0])]
        assert(line[0] == len(polygon[i+1])) # correct number of vertices 
    assert(nPoly == len(polygon)) # correct number of polygons
    return start, goal, polygon

In [0]:
def getStates(start, goal, polygon):
    """Returns the list of states in the problem:
    'start', 'goal', and the vertices of the polygons are states"""
    states = [start, goal]
    for p in polygon: states += polygon[p]
    return states

"Functions to compute distance between states:"

In [0]:
def distance(point1, point2):
    """Returns distance between two points"""
    return ((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)**0.5

"Utilities used in getSucessors function"

In [0]:
def chords(poly):
    """Returns list of chords that prevent movement through a polygon
    A polygon with n vertices has n chords (quadrilateral has 2 chords)
    A line segment joining vertices i and i+2 is a chord"""
    n = len(poly)
    if n == 4: return [(poly[0], poly[2]), (poly[1], poly[3])] # quadrilateral
    return [(poly[i%n], poly[(i+2)%n]) for i in range(n)]

In [0]:
def generateAllChords(polygon):
    """Returns list of line segments that prevent movement through polygons"""
    chordList = []
    for p in polygon: chordList += chords(polygon[p])
    return chordList

In [0]:
def intersect(ls1, ls2):
    """Returns TRUE if line segments ls1 and ls2 intersect """
    p1x, p1y, p2x, p2y = ls1[0][0], ls1[0][1], ls1[1][0], ls1[1][1]
    q1x, q1y, q2x, q2y = ls2[0][0], ls2[0][1], ls2[1][0], ls2[1][1]
    k1, k2, k3 = p1x - p2x, q2y - q1y, p1y - p2y
    k4, k5, k6 = q2x - q1x, p1x - q1x, p1y - q1y
    d = k1*k2 - k3*k4
    if d == 0:
        return False
    a, b = 1.0*(k2*k5 - k4*k6)/d, 1.0*(k1*k6 - k3*k5)/d
    if a>0 and a<1 and b>0 and b<1:
        return True
    return False

In [0]:
def visible(point1, point2, chordList):
    """Returns True if point1 visible from point2"""
    return all(not(intersect((point1, point2), chord)) for chord in chordList) 


"Problem specific functions used in Algorithm A*"

In [0]:
def getSuccessors(state, states, chordList):
    """Returns a list of states directly reachable from a given state."""
    return [v for v in states if v != state and visible(v, state, chordList)]

In [0]:
def heuristicCost(state, goal):
    """Returns distance from state to goal in the absence of obstacles"""
    return distance(state, goal)

In [0]:
def pathCost(path):
    """Returns path cost"""
    return sum([distance(path[i],path[i+1]) for i in range(len(path)-1)])

In [0]:
def Astar(start, goal, states, chordList):
    """Returns lowest cost path from "start" to "goal" using Algorithm A*.
    Partial paths from start to goal maintained in a priority queue based on 
    f(n) = g(n) + h(n) where
    - g(n) is the cost of the path from start to state n, and
    - h(n) is the heuristic cost of reaching the goal from state n.
    Assumes consistent heuristic.    
    Uses problem specific functions: heuristicCost, pathCost, getSuccessors.
    """
    priorityQ = [(heuristicCost(start, goal), [start])] # initialize queue
    explored = [] # initialize list of explored states
    while priorityQ: # paths remain to be explored
        path = heapq.heappop(priorityQ)[1] # path with lowest f(n)
        state = path[-1] # terminal state of path
        explored += [state] # add state to explored
        if state == goal: return path # goal found
        # Goal not yet found, so extend path
        successors = getSuccessors(state, states, chordList) # get successors
        for s in successors: # for each successor
            if s not in explored: # avoid explored states
                newPath = path + [s] # extend path
                fValue = pathCost(newPath) + heuristicCost(s, goal) 
                heapq.heappush(priorityQ, (fValue, newPath)) # add newPath 
    print("Goal not found") 
    return

In [0]:
def saveFigure(polygon, start, goal, path, outDir, filename):
    """Saves figure: shortest path around convex polygons"""
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, xlim=(0, 1000), ylim=(0, 1000))
    for p in polygon:
        poly = plt.Polygon(polygon[p], color = "red", ec = "black")
        ax.add_patch(poly)
    poly = plt.Polygon(path, ec="black", fill=False, closed=False, linewidth=2)
    ax.add_patch(poly)
    plt.title(filename)
    fig.savefig(outDir+filename+'.png')   # save the figure to file
    plt.close(fig)
    return

In [0]:
def solve(dataDir, outDir, filename):
    """Displays lowest cost path from start to goal
    Writes solution into file 'robot.out.txt'
    """
    sTime = time()
    start, goal, polygon = readProblem(dataDir+filename)
    print("Input file: %s" %filename)
    print("Start: %s, Goal: %s" %(str(start), str(goal)))
    states = getStates(start, goal, polygon)
    chordList = generateAllChords(polygon)
    path = Astar(start, goal, states, chordList)
    runTime = time() - sTime
    print("Run time: %0.4f seconds" % runTime)
    pCost = pathCost(path)
    print("Path cost: %0.2f " % pCost)
    ps = ', '.join([str(v) for v in path])
    print("Path: " + ps)
    saveFigure(polygon, start, goal, path, outDir, filename)
    return [filename, pCost, ps]

In [17]:
result = []
for fn in inputFiles:
    result.append(solve(dataDir, outDir, fn))

# save results to CSV file
result = pd.DataFrame(result, columns=['problem', 'cost', 'path'])
result.to_csv(outDir+'HW2Q1results.csv')

Input file: robot1.txt
Start: (69, 111), Goal: (858, 918)
Run time: 0.7835 seconds
Path cost: 1134.59 
Path: (69, 111), (215, 278), (284, 321), (305, 350), (342, 394), (372, 411), (384, 421), (472, 511), (507, 565), (615, 678), (858, 918)
